In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.layers import Concatenate
from keras import backend as K
import keras

把一个样本的标签集表示为类标签依赖矩阵。m(i,j)=1,当y(i)=1且y(j)=1；m(i,j)=0,其它

In [2]:
def dependentMatrix(y):
    n = len(y)
    m = np.zeros(shape=[n,n])
    for i in range(n):
        for j in range(n):
            if y[i] == 1 and y[j] == 1:
                m[i][j] = 1
    return m

In [3]:
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
data = np.load('AMPs-ML.npz')
X = data['X']
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=42)

In [4]:
n_train = X_train.shape[0]
label_train = np.ndarray([n_train,6,6,1])
for i in range(n_train):
    label_train[i,:,:,0] = dependentMatrix(y_train[i])

In [6]:
def model():
    chanDim = -1
    input1 = Input(shape=[20,20,3])
    out1 = Conv2D(32,(3,3), padding="same", activation='relu')(input1)
    out1 = BatchNormalization(axis=chanDim)(out1)
    out1 = MaxPooling2D(pool_size=(3, 3))(out1)
    out1 = Dropout(0.25)(out1)

    # (CONV => RELU) * 2 => POOL
    out1 = Conv2D(64, (3, 3), padding="same",activation='relu')(out1)
    out1 = BatchNormalization(axis=chanDim)(out1)
    out1 = Conv2D(64, (3, 3), padding="same",activation='relu')(out1)
    out1 = BatchNormalization(axis=chanDim)(out1)
    out1 = MaxPooling2D(pool_size=(2, 2))(out1)
    out1 = Dropout(0.25)(out1)

    # (CONV => RELU) * 2 => POOL
    out1 = Conv2D(128, (3, 3), padding="same",activation='relu')(out1)
    out1 = BatchNormalization(axis=chanDim)(out1)
    out1 = Conv2D(128, (3, 3), padding="same",activation='relu')(out1)
    out1 = BatchNormalization(axis=chanDim)(out1)
    out1 = MaxPooling2D(pool_size=(2, 2))(out1)
    out1 = Dropout(0.25)(out1)

    # first set of FC => RELU layers
    out1 = Flatten()(out1)
    out1 = Dense(1024, activation='relu')(out1)
    out1 = BatchNormalization()(out1)
    out1 = Dropout(0.5)(out1)
    
    # second set of FC
    out1 = Dense(2048, activation='relu')(out1)
    out1 = BatchNormalization()(out1)
    out1 = Dropout(0.5)(out1)
    
    model1 = Model(input1, out1)
    
    input2 = Input(shape=[6,6,1])
    out2 = Conv2D(32,(3,3), padding="same", activation='relu')(input2)
    out2 = BatchNormalization(axis=chanDim)(out2)
    
    out2 = Conv2D(64,(3,3), padding="same", activation='relu')(input2)
    out2 = BatchNormalization(axis=chanDim)(out2)
    
    out2 = Conv2D(128,(3,3), padding="same", activation='relu')(input2)
    out2 = BatchNormalization(axis=chanDim)(out2)
    
    out2 = Flatten()(out2)
    out2 = Dense(512, activation='relu')(out2)
    out2 = BatchNormalization()(out2)
    out2 = Dropout(0.5)(out2)
    
   
    out2 = Dense(1024, activation='relu')(out2)
    out2 = BatchNormalization()(out2)
    out2 = Dropout(0.5)(out2)
    
    model2 = Model(input2,out2)
    
    conc = Concatenate(axis=-1)([model1.output,model2.output])
    out = Dense(1024, activation="relu")(conc)
    out = Dropout(rate=0.5)(out)
    out = Dense(units=6, activation="sigmoid")(out)
    
    model = Model([model1.input, model2.input], out)
    
    return model

In [9]:
INIT_LR = 1e-3
EPOCHS = 75
BS = 32
# initiallize the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
nn = model()
nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli distribution

# train the network
print("[INFO] training network...")

nn.fit([X_train, label_train], y_train,
          epochs=20,
          batch_size=64,
         verbose=0)
score = nn.evaluate(X_test, y_test, batch_size=64)

[INFO] training network...


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[[[0.        , 0.        , 0.        ],
         [0.76541968, 0.        , 0.        ],
         [0.49257244, 0.        , 0.        ],
         ...,
         [0.57705951, 0.        , 0.        ...

In [ ]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # use a *softmax* activation for single-label classification
        # and *sigmoid* activation for multi-label classification
        model.add(Dense(classes))
        model.add(Activation(finalAct))

        # return the constructed network architecture
        return model

In [ ]:
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
data = np.load('AMPs-ML.npz')
X = data['X']
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

In [ ]:

y1 = y_train[:,0]

y1_cate = to_categorical(y1,num_classes=2)
sum(y1_cate)

In [ ]:
# 分成6个2分类预测器，每个预测器预测一个标签
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss

INIT_LR = 1e-3
EPOCHS = 20
BS = 32
y_pred=np.zeros([y_test.shape[0],6])
for i in range(6):
    y_sub = y_train[:,i]
    y_bin = to_categorical(y_sub, num_classes=2)
    y_count = sum(y_bin)
    cw = {0: 1/(2*y_count[0]), 1: 1/(2*y_count[1])}
    model = SmallerVGGNet.build(width=20, height=20, depth=3, classes=2, finalAct='softmax')
    opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    print('[INFO] training network for label {}'.format(i))
    model.fit(X_train,y_bin, batch_size=BS, epochs=20,
             class_weight=cw, verbose=0)
    y_pred[:,i] = model.predict_classes(X_test)


print("the subset accuracy: {}".format(accuracy_score(y_test,y_pred)))
print("hamming loss: {}".format(hamming_loss(y_test,y_pred)))

In [ ]:
accuracy_score(y_test[:,5],y_pred[:,5])

In [ ]:
#同时预测6个标签
import numpy as np
data = np.load('AMPs-ML.npz')
X = data['X']
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=42)

INIT_LR = 1e-3
EPOCHS = 75
BS = 32
model = SmallerVGGNet.build(width=20, height=20, depth=3, classes=6, finalAct="sigmoid")

# initiallize the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
w = sum(y_train)
cw = {}
for i in range(6):
    cw[i] = 1/(6*w[i])
# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli distribution
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")
model.fit(X_train, y_train,
          epochs=20,
          batch_size=64,
         class_weight=cw,
         verbose=0)
score = model.evaluate(X_test, y_test, batch_size=64)

In [ ]:
p = model.predict(X_test)
#print("the subset accuracy: {}".format(accuracy_score(y_test,p)))
#print("hamming loss: {}".format(hamming_loss(y_test,p)))

In [ ]:
y_test.shape

In [ ]:
p=dependentMatrix(y_test[-2])
p

In [ ]:
label_train.shape


In [ ]:
X_train.shape
